In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from tqdm.notebook import tqdm

import os

In [2]:
# df_data = pd.read_csv("../Dataset/processedDataWithThaliaAdded.csv", nrows=10000,delimiter='|')
df_data = pd.read_csv("../Dataset/processedDataWithThaliaAdded.csv", sep = "|")
df_data.head(5)

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,5649,... and the Word became a Story,... and the Word became a Story,and the word became a story,The Author,The Author,the author,en,Englisch,2.0,...,Books on Demand,BoD – Books on Demand,2018-04-17,At the beginning was the Word. And from the Wo...,FL,FL FM FN,http://books.google.com/books/content?id=FrElD...,NOT_MATURE,304.0,False
1,77497,... EVVAI!,... EVVAI!,evvai,Cristina Polacchini,CRISTINA POLACCHINI,cristina polacchini,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2015-08-01,"Amicizia, scuola, amore, alti e bassi, in and ...",YFB,,http://books.google.com/books/content?id=lGNgC...,NOT_MATURE,106.0,False
2,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,NaN,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,NaN,2008,NaN,YFM,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,False
3,24800,... trägt Jeans und Tennisschuhe,"""... trägt Jeans und Tennisschuhe""",traegt jeans und tennisschuhe,Frauke Kühn,Frauke Kühn,frauke kuehn,un,Deutsch,2.0,...,Rowohlt Repertoire,NaN,1995,NaN,YFCF,5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...,NaN,NOT_MATURE,7.0,False
4,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False


In [3]:
df_dataset = df_data[['itemID','title_thalia','author_thalia','publisher_api','main topic','subtopics']]
df_dataset.fillna("",inplace=True)

df_dataset['metadata'] = df_dataset[['main topic','subtopics']].apply(lambda x: ' '.join(x), axis = 1)
#df_dataset['metadata2'] = df_dataset[['author_thalia','publisher_api']].apply(lambda x: ' '.join(x), axis = 1)


df_dataset['metadata'] = df_dataset['metadata'].str.replace('|',' ')
df_dataset.head(5)


C:\Users\rmaql\Anaconda3\lib\site-packages\pandas\core\frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-3-ab1315b834c8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset['metadata'] = df_dataset[['main topic','subtopics']].apply(lambda x: ' '.join(x), axis = 1)
<ipython-input-3-ab1315b834c8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata
0,5649,and the word became a story,the author,BoD – Books on Demand,FL,FL FM FN,FL FL FM FN
1,77497,evvai,cristina polacchini,Lulu.com,YFB,,YFB
2,13238,then he ate my boy entrancers,louise rennison,,YFM,4Z GB ACN YFQ,YFM 4Z GB ACN YFQ
3,24800,traegt jeans und tennisschuhe,frauke kuehn,,YFCF,5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...,YFCF 5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...
4,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,FL,,FL


In [4]:
indices =pd.Series(df_dataset.index,index = df_dataset["title_thalia"]).drop_duplicates()
indices

title_thalia
 and the word became a story                       0
 evvai                                             1
 then he ate my boy entrancers                     2
 traegt jeans und tennisschuhe                     3
 tripion  minaccia dallo spazio                    4
                                               ...  
the wonderful adventures of nils               57454
too close for comfort                          57455
trail of lightning                             57456
virals                                         57457
warriors dawn of the clans 01 the sun trail    57458
Length: 57459, dtype: int64

In [5]:
#Recommendations based on Metadata (Topic/Subtopic)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_dataset['metadata'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [6]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation = linear_kernel(tfidf_matrix, tfidf_matrix)

MemoryError: Unable to allocate 24.6 GiB for an array with shape (57459, 57459) and data type float64

In [15]:
#Recommendations based on Metadata (Topic/Subtopic)
def get_recommendations_topic(title, cosine_sim, method_name):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top 5 most similar movies
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    book_indices = [i[0] for i in sim_scores]

    # Get the pred scores
    book_pred_score = [i[1] for i in sim_scores]

    # Return the top 5 most similar movies
    return pd.DataFrame({'itemID': book_indices,'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'metadata': list(df_dataset['metadata'].iloc[book_indices].values), method_name: book_pred_score})

In [24]:
df_dataset

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata
0,5649,and the word became a story,the author,BoD – Books on Demand,FL,FL FM FN,FL FL FM FN
1,77497,evvai,cristina polacchini,Lulu.com,YFB,,YFB
2,13238,then he ate my boy entrancers,louise rennison,,YFM,4Z GB ACN YFQ,YFM 4Z GB ACN YFQ
3,24800,traegt jeans und tennisschuhe,frauke kuehn,,YFCF,5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...,YFCF 5AN 1DFG 1QBDN YFB YFY YXH YXJ 3MPQX ...
4,26966,tripion minaccia dallo spazio,stefano grimaldi,Lulu.com,FL,,FL
...,...,...,...,...,...,...,...
95,4795,13 chilling tales,edwin f becker,AuthorHouse,DNT,FM,DNT FM
96,31462,13 gifts a wish novel,wendy mass,Scholastic Paperbacks,YFH,,YFH
97,60335|69287|55553,13 kings,"v s nesby, vs nesby",Xlibris Corporation,FL,,FL
98,45341,13 little blue envelopes epic reads edition,maureen johnson,Katherine Tegen Books,YF,YX,YF YX


In [23]:
book_search = 'kings'
display(df_dataset[df_dataset['title_thalia']==book_search])

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata


In [20]:
book_search = 'evvai'
display(df_dataset[df_dataset.title_thalia==book_search])
get_recommendations_topic("evvai",content_correlation, 'Predicted Rating')

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata


KeyError: 'evvai'

In [10]:
# Recommendations Based of Author
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix2 = tfidf.fit_transform(df_dataset['author_thalia'])
tfidf_df2 = pd.DataFrame(tfidf_matrix.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [23]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)

In [24]:
#Recommendations based on Author
def get_recommendations_author(title, cosine_sim, method_name):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top 5 most similar movies
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    book_indices = [i[0] for i in sim_scores]

    # Get the pred scores
    book_pred_score = [i[1] for i in sim_scores]

    return pd.DataFrame({'itemID': book_indices,'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'author_thalia': list(df_dataset['author_thalia'].iloc[book_indices].values), method_name: book_pred_score})

In [28]:
book_search = 'famous'
display(df_dataset[df_dataset.title_thalia==book_search])
get_recommendations_author(book_search,content_correlation2, 'Predicted Rating')

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata
1,32836,famous,"jilly gagnon, todd strasser",Planet!,YFM,YXQ,YFM YXQ


,itemID,title_thalia,author_thalia,Predicted Rating
0,5386,blood on my hands,todd strasser,0.663472
1,6420,cant get there from here,todd strasser,0.663472
2,9953,date with a rockstar,sarah gagnon,0.432974
3,4668,belle and moozie and whatshisname,gayle strasser,0.349040
4,8281,cuenta la historia de la quiropráctica,todd waters,0.307443


In [30]:
# Recommendations Based of Publisher
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix3 = tfidf.fit_transform(df_dataset['publisher_api'])
tfidf_df3 = pd.DataFrame(tfidf_matrix.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [32]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation3 = linear_kernel(tfidf_matrix3, tfidf_matrix3)

In [36]:
def get_recommendations_publisher(title, cosine_sim, method_name):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top 5 most similar movies
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    book_indices = [i[0] for i in sim_scores]

    # Get the pred scores
    book_pred_score = [i[1] for i in sim_scores]

    return pd.DataFrame({'itemID': book_indices,'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'publisher_api': list(df_dataset['publisher_api'].iloc[book_indices].values), method_name: book_pred_score})

In [37]:
book_search = 'famous'
display(df_dataset[df_dataset.title_thalia==book_search])
get_recommendations_publisher(book_search,content_correlation3, 'Predicted Rating')

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,metadata
1,32836,famous,"jilly gagnon, todd strasser",Planet!,YFM,YXQ,YFM YXQ


,itemID,title_thalia,publisher_api,Predicted Rating
0,6236,cappuccetto rosso,The Planet,1.0
1,0,tripion minaccia dallo spazio,Lulu.com,0.0
2,2,grounded at fort irwin,Xlibris Corporation,0.0
3,3,its not ok,Xlibris Corporation,0.0
4,4,then he ate my boy entrancers,,0.0
